In [ ]:
# importing the neccesary packages
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from sklearn.model_selection import train_test_split
import keras
import numpy as np
import pandas as pd 
import os
import cv2
import matplotlib.pyplot as plt

In [ ]:
data_path = os.listdir("/path")

In [ ]:
sample_img = plt.imread("/path")

In [ ]:
lbl  = os.listdir("/path")
def fc_yawn(direc="/path", face_cas_path="/path"):
    y_no = []
    imgSize = 145
    cati = ["yawn", "no_yawn"]
    for cat in cati:
        p_link = os.path.join(direc, cat)
        num = cati.index(cat)
        for image in os.listdir(p_link):
            img_ar = cv2.imread(os.path.join(p_link, image), cv2.IMREAD_COLOR)
            fac_cas = cv2.CascadeClassifier(face_cas_path)
            faces = fac_cas.detectMultiScale(img_ar, 1.3, 5)
            for (x, y, w, h) in faces:
                img_es = cv2.rectangle(img_ar, (x, y), (x+w, y+h), (0, 255, 0), 2)
                roi_clr = img_es[y:y+h, x:x+w]
                re_ar = cv2.resize(roi_clr, (imgSize, imgSize))
                y_no.append([re_ar, num])
    return y_no


In [ ]:
yawn_no_yawn = fc_yawn()
def getData(dir_path="/path", face_cas="/path", eye_cas="/path"):
    lbl = ['Closed', 'Open']
    imgSize = 145
    d = []
    for lb in lbl:
        path = os.path.join(dir_path, lb)
        cls_num = lbl.index(lb)
        cls_num +=2
        for img_es in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img_es), cv2.IMREAD_COLOR)
                re_ar = cv2.resize(img_array, (imgSize, imgSize))
                d.append([re_ar, cls_num])
            except Exception as e:
                print(e)
    return d


In [ ]:
data_train = getData()
def ap_data():
    y_no = face_yawn()
    d = getData()
    y_no.extend(d)
    return np.array(y_no)
new_data = ap_data()


In [ ]:
X = []
y = []
for fea, lb in new_data:
    X.append(fea)
    y.append(lb)
X = np.array(X)
X = X.reshape(-1, 145, 145, 3)
lbling = LabelBinarizer()
y = lbling.fit_transform(y)
y = np.array(y)


In [ ]:
seed = 42
testdata_size = 0.30
xtrain, xtest, ytrain, ytest = train_test_split(X, y, random_state=seed, testdata_size=testdata_size)
train_gen = ImageDataGenerator(rescale=1/255, zoom_range=0.2, horizontal_flip=True, rotation_range=30)

In [ ]:
test_gen = ImageDataGenerator(rescale=1/255)

train_gen = train_gen.flow(np.array(xtrain), ytrain, shuffle=False)
test_gen = test_gen.flow(np.array(xtest), ytest, shuffle=False)


In [ ]:
model_def = Sequential()

model_def.add(Conv2D(256, (3, 3), activation="relu", input_shape=xtrain.shape[1:]))
model_def.add(MaxPooling2D(2, 2))

model_def.add(Conv2D(128, (3, 3), activation="relu"))
model_def.add(MaxPooling2D(2, 2))

model_def.add(Conv2D(64, (3, 3), activation="relu"))
model_def.add(MaxPooling2D(2, 2))

model_def.add(Conv2D(32, (3, 3), activation="relu"))
model_def.add(MaxPooling2D(2, 2))

model_def.add(Flatten())
model_def.add(Dropout(0.5))

model_def.add(Dense(64, activation="relu"))
model_def.add(Dense(4, activation="softmax"))

model_def.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

model_def.summary()


In [ ]:
History = model_def.fit(train_gen, epochs=25, validation_data=test_gen, shuffle=True, validation_steps=len(test_gen))

In [ ]:
accuracy = History.History ['accuracy']
val_accuracy = History.History ['val_accuracy']
loss = History.History['loss']
val_loss = History.History['val_loss']
epochs = range(len(accuracy))

plt.plot(epochs, accuracy, "g", lb="trainning accuracy")
plt.plot(epochs, val_accuracy, "r", lb="validation accuracy")
plt.legend()
plt.show()

plt.plot(epochs, loss, "g", lb="trainning loss")
plt.plot(epochs, val_loss, "r", lb="validation loss")
plt.legend()
plt.show()


In [ ]:
model_def.save("dorwsii.h5")

In [ ]:
prediction=np.argmax(model_def.predict(xtest), axis=-1)
labels_new = ["yawn", "no_yawn", "Closed", "Open"]
imgSize = 145
def predict(filepath, face_cas="/path"):
    img_array = cv2.imread(filepath, cv2.IMREAD_COLOR)
    img_array = img_array / 255
    re_ar = cv2.resize(img_array, (imgSize, imgSize))
    return re_ar.reshape(-1, imgSize, imgSize, 3)

model_pre = tf.keras.models.load_model("C:/Users/91915/dorwsii.h5")


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(np.argmax(ytest, axis=1), prediction, target_names=labels_new))
prediction = model_pre.predict([predict("H:/UK/Ann/testing/test5cl.jpg")])
np.argmax(prediction)
